In [ ]:
import unittest
from platform import python_version
import importlib
import toml

def parse_pyproject_toml():
    with open('pyproject.toml', 'r') as file:
        pyproject_data = toml.load(file)

    dependencies = pyproject_data.get('tool', {}).get('poetry', {}).get('dependencies', {})
    dependencies.pop('python', None)

    return dependencies

def parse_poetry_lock():
    with open('poetry.lock', 'r') as file:
        lock_data = toml.load(file)

    dependencies = {}
    for package in lock_data.get('package', []):
        name = package['name']
        version = package['version']
        dependencies[name] = version

    return dependencies

def parse_python_version():
    with open('.python-version', 'r') as file:
        python_version = file.read()
    file.close()

    return python_version

def get_major_minor(s):
    return '.'.join(s.split('.')[:2])

class CommonTests(unittest.TestCase):
    def test_python_version(self):
        expected_major_minor = parse_python_version()
        actual_major_minor = get_major_minor(python_version())
        self.assertEqual(actual_major_minor, expected_major_minor, "incorrect python version")

    def test_dependencies_versions(self):
        direct_dependencies_map = parse_pyproject_toml()
        all_dependencies_map = parse_poetry_lock()

        for package, defined_version in direct_dependencies_map.items():
            expected_version = all_dependencies_map[package]
            installed_version = importlib.metadata.version(package)
            self.assertEqual(installed_version, expected_version, f"incorrect {package} version")

unittest.main(argv=[''], verbosity=2, exit=False)